In [ ]:
using Revise

In [ ]:
using Polyhedra
using CDDLib
using DrakeVisualizer
DrakeVisualizer.any_open_windows() || DrakeVisualizer.new_window()
using RigidBodyDynamics
using StaticArrays
using RigidBodyTreeInspector
using LCPSim
using JuMP, Gurobi
using Base.Test
using ProfileView

In [ ]:
function DrakeVisualizer.addgeometry!(vis::Visualizer, obs::Obstacle, boundary::HRepresentation=SimpleHRepresentation(vcat(eye(3), -eye(3)), vcat([2.5, 0.5, 2.0], -[-1., -0.5, -0.1])))
    p = intersect(boundary, obs.interior)
    addgeometry!(vis, CDDPolyhedron{3, Float64}(p))
end

In [ ]:
urdf_mech = parse_urdf(Float64, "box_valkyrie.urdf")
mechanism, base = planar_revolute_base()
attach!(mechanism, base, urdf_mech)
world = root_body(mechanism)

basevis = Visualizer()[:boxval]
delete!(basevis)
vis = basevis[:robot]
setgeometry!(vis, mechanism, parse_urdf("box_valkyrie.urdf", mechanism))

floor = planar_obstacle(default_frame(world), [0, 0, 1.], [0, 0, 0.], 1.)
wall = planar_obstacle(default_frame(world), [1., 0, 0], [-0.7, 0, 0], 1.)

addgeometry!(basevis[:environment], wall)
addgeometry!(basevis[:environment], floor)

contact_limbs = findbody.(mechanism, ["rh", "lh", "rf", "lf"])
hands = findbody.(mechanism, ["rh", "lh"])
feet = findbody.(mechanism, ["rf", "lf"])

env = Environment(
    Dict(vcat(
            [body => ContactEnvironment(
                [Point3D(default_frame(body), SVector(0., 0, 0))],
                [floor])
                for body in feet],
            [body => ContactEnvironment(
                [Point3D(default_frame(body), SVector(0., 0, 0))],
                [wall])
                for body in [findbody(mechanism, "lh")]]
            )));

In [ ]:
inspect(mechanism, vis)

In [ ]:
function nominal_input(x0::MechanismState)
    u_nominal = clamp.(inverse_dynamics(x0, zeros(num_velocities(x0))), LCPSim.all_effort_bounds(x0.mechanism))
    feet = findbody.(x0.mechanism, ["rf", "lf"])
    weight = mass(x0.mechanism) * mechanism.gravitational_acceleration.v[3]
    u_nominal[parentindexes(velocity(x0, findjoint(x0.mechanism, "core_to_rf_z")))...] += weight / 2
    u_nominal[parentindexes(velocity(x0, findjoint(x0.mechanism, "core_to_lf_z")))...] += weight / 2
    u_nominal
end

In [ ]:
x0 = MechanismState{Float64}(mechanism)
set_velocity!(x0, zeros(num_velocities(x0)))
set_configuration!(x0, findjoint(mechanism, "base_x"), [0])
set_configuration!(x0, findjoint(mechanism, "base_z"), [0.9])
# set_velocity!(x0, findjoint(mechanism, "core_to_rf_x"), [-1])
set_configuration!(x0, findjoint(mechanism, "core_to_rh_x"), [0.6])
set_configuration!(x0, findjoint(mechanism, "core_to_rh_z"), [0.0])
set_configuration!(x0, findjoint(mechanism, "core_to_lh_x"), [0.6])
set_configuration!(x0, findjoint(mechanism, "core_to_lh_z"), [0.0])
set_configuration!(x0, findjoint(mechanism, "core_to_rf_x"), [0.2])
set_configuration!(x0, findjoint(mechanism, "core_to_rf_z"), [-0.9])
set_configuration!(x0, findjoint(mechanism, "core_to_lf_x"), [0.2])
set_configuration!(x0, findjoint(mechanism, "core_to_lf_z"), [-0.9])

settransform!(vis, x0)
q0 = copy(configuration(x0))
v0 = copy(velocity(x0))
u0 = nominal_input(x0)

contacts = [Point3D(default_frame(body), SVector(0., 0, 0)) for body in feet]
qq = [100, 0.1, 1, fill(0.1, num_positions(x0) - 3)...]
Q = diagm(vcat(qq, fill(0.1, num_velocities(x0))))
R = 0.001 * eye(num_velocities(x0))
K = LCPSim.ContactLQR.contact_lqr(x0, u0, Q, R, contacts)

controller = x -> begin
    -K * (state_vector(x) - vcat(q0, v0)) .+ u0
end

Δt = 0.01

set_velocity!(x0, findjoint(mechanism, "base_x"), [-1.0])
@time results = LCPSim.simulate(x0, controller, env, Δt, 100, GurobiSolver(OutputFlag=0));
# Profile.clear()
# @profile LCPSim.simulate(x0, controller, env, Δt, 15, GurobiSolver(OutputFlag=0));
# ProfileView.view()

In [ ]:
for r in results
    set_configuration!(x0, configuration(r.state))
    settransform!(vis, x0)
    sleep(Δt)
end

In [ ]:
function run_optimization(x0, env, Δt, N; x_nominal=x0, solver=GurobiSolver())
    qstar = configuration(x_nominal)
    vstar = zeros(num_velocities(x_nominal))
    ustar = nominal_input(x_nominal)
    
    model, results_opt = LCPSim.optimize(x0, env, Δt, N, Model(solver=solver))
    @show configuration(x_nominal)

    w = 0.1 * ones(num_positions(x0))
    w[1] = 0
    w[2] = 10
    w[3] = 1
    w[6:7] = 1
    
    objective = (
    0
#         + 10 * sum((configuration(results_opt[end].state)[2:end] .- q0[2:end]).^2)
#         + 10 * sum((velocity(results_opt[end].state)[1:3]).^2)
#         + 10 * sum((configuration(results_opt[end].state)[1:2] .- [0, 1]).^2)
#         + 100 * configuration(results_opt[end].state)[3]^2
#         + 10 * sum((configuration(results_opt[end].state)[[6, 7, 10, 11]] .- [0.2, 0.2, -0.9, -0.9]).^2)
        + 10 * sum(w .* (qstar .- configuration(results_opt[end].state)).^2)
#             + sum(velocity(results_opt[end].state).^2)
        )
    
    w[6:7] = 0.1
    w[2] = 1
    
    for r in results_opt
#         @variable model y[1:length(r.input)]
#         @constraints model
#             y .>= r.input
#             y .>= .-r.input
#         end
        objective += (
            0
#             + 0.1 * sum(y)
            + 0.0001 * sum((r.input).^2)
            + 0.1 * sum(w .* (qstar .- configuration(r.state)).^2)
#             + 1 * (q0[1] - configuration(r.state)[1])^2
#             + (q0[1] - configuration(r.state)[1])^2
            + 0.1 * velocity(r.state)[1]^2
#             + 0.1 * (configuration(r.state)[6] - configuration(r.state)[7])^2
#             + 10 * sum((configuration(r.state)[2:end] .- q0[2:end]).^2)
#             + 1 * sum(velocity(r.state)[1:3].^2)
#             + 0.1 * sum(velocity(r.state)[4:end].^2)
#             + configuration(r.state)[3]^2
#             + sum((q0 .- configuration(r.state)).^2)
#             + 0.01 * sum(velocity(r.state).^2)
            )
            
    end
    
#     for i in 2:length(results_opt)
#         objective += (
#             0.1 * sum((velocity(results_opt[i].state) .- velocity(results_opt[i].state)).^2)
#             )
#     end


    # for r in results_opt
    #     @constraint model configuration(r.state)[3] <= π/16
    #     @constraint model configuration(r.state)[3] >= -π/16
    # end

    for r in results_opt
        for (joint, jrs) in r.joint_contacts
            for joint_result in jrs
#                 JuMP.fix(joint_result.λ, 0)
                objective += joint_result.λ^2
            end
        end 
    end
    
    @objective model Min objective

    
    contacts = [Point3D(default_frame(body), SVector(0., 0, 0)) for body in feet]
    qq = [100, 0.1, 1, fill(0.1, num_positions(x0) - 3)...]
    Q = diagm(vcat(qq, fill(0.1, num_velocities(x0))))
    R = 0.001 * eye(num_velocities(x0))
    K = LCPSim.ContactLQR.contact_lqr(MechanismState(x0.mechanism, qstar, vstar), 
            ustar, Q, R, contacts)

    controller = x -> begin
        -K * (state_vector(x) - vcat(qstar, vstar)) .+ ustar
    end
    Δt_sim = 0.01
    time_ratio = convert(Int, Δt / Δt_sim)
    results = LCPSim.simulate(x0, controller, env, Δt_sim, time_ratio * N, GurobiSolver(OutputFlag=0))
    for r in results
        set_configuration!(x0, configuration(r.state))
        settransform!(vis, x0)
        sleep(Δt_sim)
    end
#     setvalue.(results_opt, results[1:time_ratio:end])
#     ConditionalJuMP.warmstart!(model, false)
    
    solve(model)
    getvalue.(results_opt)
end

In [ ]:
x0 = MechanismState{Float64}(mechanism)
set_velocity!(x0, zeros(num_velocities(x0)))
set_velocity!(x0, findjoint(mechanism, "base_x"), [-3])
set_velocity!(x0, findjoint(mechanism, "base_rotation"), [0.])
set_configuration!(x0, findjoint(mechanism, "base_x"), [0.75])
set_configuration!(x0, findjoint(mechanism, "base_z"), [.9])
set_configuration!(x0, findjoint(mechanism, "core_to_rh_x"), [0.6])
set_configuration!(x0, findjoint(mechanism, "core_to_rh_z"), [0.0])
set_configuration!(x0, findjoint(mechanism, "core_to_lh_x"), [0.6])
set_configuration!(x0, findjoint(mechanism, "core_to_lh_z"), [0.0])
set_configuration!(x0, findjoint(mechanism, "core_to_rf_x"), [0.2])
set_configuration!(x0, findjoint(mechanism, "core_to_rf_z"), [-.9])
set_configuration!(x0, findjoint(mechanism, "core_to_lf_x"), [0.2])
set_configuration!(x0, findjoint(mechanism, "core_to_lf_z"), [-.9])
q0 = copy(configuration(x0))
v0 = copy(velocity(x0))
u0 = nominal_input(x0)

xstar = MechanismState(mechanism, copy(q0), zeros(v0))

In [ ]:
set_configuration!(x0, q0)
set_velocity!(x0, v0)
Δt = 0.05
gap = 0.001
timelimit = 60

r_control = []
x_control = MechanismState(mechanism, copy(q0), copy(v0))
controller = x -> begin
    set_velocity!(x_control, velocity(x))
    set_configuration!(x_control, configuration(x))
    r_control = run_optimization(x_control, env, Δt, 15; x_nominal=xstar, solver=GurobiSolver(OutputFlag=0, MIPGap=gap, TimeLimit=timelimit))
    u = r_control[1].input
    @show u
end

results = LCPSim.simulate(x0, controller, env, Δt, 15, GurobiSolver(OutputFlag=0));

In [ ]:
set_configuration!(x0, q0)
settransform!(vis, x0)

In [ ]:
set_configuration!(x0, q0)
settransform!(vis, x0)
for r in results
    sleep(Δt)
    set_configuration!(x0, configuration(r.state))
    settransform!(vis, x0)
    for (body, contacts) in r.contacts
        for (i, contact) in enumerate(contacts)
            f = LCPSim.contact_force(contact)
            p = transform_to_root(x0, contact.point.frame) * contact.point
            v = vis[:forces][Symbol(body)][Symbol(i)]
            setgeometry!(v, DrakeVisualizer.PolyLine([p.v, (p + 0.1*f).v]; end_head=DrakeVisualizer.ArrowHead()))
        end
    end
end

In [ ]:
set_configuration!(x0, configuration(results[end].state))
set_velocity!(x0, velocity(results[end].state))

r_control = []
x_control = MechanismState(mechanism, copy(q0), copy(v0))
controller = x -> begin
    set_velocity!(x_control, velocity(x))
    set_configuration!(x_control, configuration(x))
    r_control = run_optimization(x_control, env, Δt, 10; x_nominal=xstar, solver=GurobiSolver(OutputFlag=0, MIPGap=gap, TimeLimit=timelimit))
    u = r_control[1].input
    @show u
end

append!(results, LCPSim.simulate(x0, controller, env, Δt, 15, GurobiSolver(OutputFlag=0)));

In [ ]:
set_configuration!(x0, q0)
set_velocity!(x0, v0)
Δt = 0.05
results_opt = run_optimization(x0, env, Δt, 15, solver=GurobiSolver(MIPGap=0.01, TimeLimit=60));

In [ ]:
set_configuration!(x0, q0)
settransform!(vis, x0)
for r in results_opt
    sleep(Δt)
    set_configuration!(x0, configuration(r.state))
    settransform!(vis, x0)
end

In [ ]:
set_configuration!(x0, configuration(results_opt[end].state))
set_velocity!(x0, velocity(results_opt[end].state))
append!(results_opt, run_optimization(x0, env, Δt, 15, solver=GurobiSolver(MIPGap=0.01, TimeLimit=60)));

set_configuration!(x0, q0)
settransform!(vis, x0)
for r in results_opt
    sleep(Δt)
    set_configuration!(x0, configuration(r.state))
    settransform!(vis, x0)
end

In [ ]:
x0 = MechanismState(mechanism, copy(q0), copy(v0))
u_nominal = nominal_input(x0)
i = 1
controller = x -> begin
    global i
    if i <= length(results_opt)
        i += 1
        results_opt[i - 1].input
    else
        kp = 20
        kd = 0.1 * kp
        kp .* (q0 .- configuration(x)) .- kd .* velocity(x) .+ u_nominal
    end
        
end

results = LCPSim.simulate(x0, controller, env, Δt, length(results_opt), GurobiSolver(OutputFlag=0));

In [ ]:
set_configuration!(x0, q0)
settransform!(vis, x0)
for r in results
    sleep(Δt)
    set_configuration!(x0, configuration(r.state))
    settransform!(vis, x0)
end

In [ ]:
x0 = MechanismState(mechanism, copy(q0), copy(v0))
model, results_to_warm = LCPSim.optimize(x0, env, Δt, results[1:30], Model(solver=GurobiSolver()))

objective = (
    0
        + 0.1 * sum((configuration(results_opt[end].state)[2:end] .- q0[2:end]).^2)
        + 0.01 * sum((velocity(results_opt[end].state)).^2)
#         + 10 * sum((configuration(results_opt[end].state)[1:2] .- [0, 1]).^2)
        + 100 * configuration(results_opt[end].state)[3]^2
        + 10 * sum((configuration(results_opt[end].state)[[6, 7, 10, 11]] .- [0.2, 0.2, -0.9, -0.9]).^2)
        )
for r in results_to_warm
    objective += configuration(r.state)[3]^2
    objective += 0.001 * sum((r.input .- u_nominal).^2)
end
@objective model Min objective
# setvalue.(results_to_warm, results[1:length(results_to_warm)]);
ConditionalJuMP.warmstart!(model, true)

In [ ]:
solve(model)
results_warmed = getvalue.(results_to_warm);

In [ ]:
set_configuration!(x0, q0)
settransform!(vis, x0)
for r in results_warmed
    sleep(Δt)
    set_configuration!(x0, configuration(r.state))
    settransform!(vis, x0)
end

In [ ]:
vis2 = basevis[:robot2]
setgeometry!(vis2, mechanism, parse_urdf("box_valkyrie.urdf", mechanism))

In [ ]:


set_configuration!(x0, q0)
settransform!(vis, x0)
for i in 1:length(results_warmed)
    sleep(Δt)
    set_configuration!(x0, configuration(results_warmed[i].state))
    settransform!(vis, x0)
    set_configuration!(x0, configuration(results[i].state))
    settransform!(vis2, x0)
end

In [ ]:
x0 = MechanismState(mechanism, copy(q0), copy(v0))
u_nominal = nominal_input(x0)
source = results_opt
i = 1
controller = x -> begin
    global i
    if i <= length(source)
        i += 1
        source[i - 1].input
    else
        kp = 20
        kd = 0.1 * kp
        kp .* (q0 .- configuration(x)) .- kd .* velocity(x) .+ u_nominal
    end
        
end

results = LCPSim.simulate(x0, controller, env, Δt, 30, GurobiSolver(OutputFlag=0));



In [ ]:
set_configuration!(x0, q0)
settransform!(vis, x0)
for r in results
    sleep(Δt)
    set_configuration!(x0, configuration(r.state))
    settransform!(vis, x0)
end

In [ ]:
set_configuration!(x0, q0)
settransform!(vis, x0)
for r in results
    sleep(Δt)
    set_configuration!(x0, configuration(r.state))
    settransform!(vis, x0)
end